# Limpieza y Análisis de Datos con Pandas

Este notebook trabaja con el archivo `screentime_analysis_extended.csv`.

Objetivos:
1. Cargar datos y explorarlos.
2. Detectar y tratar valores nulos.
3. Eliminar duplicados.
4. Arreglar tipos de datos.
5. Limpiar texto.
6. Detectar columnas vacías y borrarlas.
7. Guardar el dataset limpio.
8. Hacer un mini análisis final.

## 1. Imports & configuración

In [ ]:
import pandas as pd
import numpy as np

## 2. Carga de datos

In [ ]:
import pandas as pd

print("=== 1) CARGANDO EL CSV DE HISTORIAL DE SPOTIFY ===")

df = pd.read_csv("spotify_history.csv", sep=";")

print("\nCSV cargado correctamente. A continuación se muestran las primeras 5 filas para ver cómo es el dataset:\n")
display(df.head())

print("\n=== 2) INFORMACIÓN GENERAL DEL DATAFRAME ===")
print("Esta tabla muestra cuántas filas tiene el dataset, cuántas columnas y el tipo de dato de cada una.\n")
df.info()

print("\n=== 3) DESCRIPCIÓN ESTADÍSTICA (NUMÉRICA) ===")
print("Aquí vemos estadísticas básicas de las columnas numéricas, como minutos reproducidos, medias, mínimos, etc.\n")
display(df.describe())

print("\n=== 4) DESCRIPCIÓN ESTADÍSTICA (INCLUYE TEXTO Y CATEGORÍAS) ===")
print("Incluye columnas con texto, categorías y valores no numéricos.\n")
display(df.describe(include='all'))


## 3. Limpieza y preparación de datos

In [ ]:
print("\n=== 3. LIMPIEZA Y PREPARACIÓN DE DATOS ===")

# 3.1 Verificar y convertir 'ts' a datetime
print("\n3.1 → Comprobando el tipo de la columna 'ts' y convirtiéndola a datetime.\n"
      "Esto permite extraer horas, días de la semana y hacer análisis temporales.")
print("Tipo ANTES:", df['ts'].dtype)

df['ts'] = pd.to_datetime(df['ts'], dayfirst=True)

print("Tipo DESPUÉS:", df['ts'].dtype)


# 3.2 Convertir ms_played a minutes_played
print("\n3.2 → Creando columna 'minutes_played' para trabajar en minutos en lugar de milisegundos.\n")
df['minutes_played'] = df['ms_played'] / 60000
display(df[['ms_played', 'minutes_played']].head())


# 3.3 Comprobar valores nulos y duplicados
print("\n3.3 → Revisando valores nulos y duplicados en el dataset.\n"
      "Esto ayuda a detectar datos incompletos o problemas de calidad.")
print("\nValores nulos por columna:")
print(df.isna().sum())

print("\nNúmero de registros duplicados:", df.duplicated().sum())


# 3.4 Convertir columnas categóricas y asegurar 0/1 en skipped/shuffle
print("\n3.4 → Convirtiendo columnas categóricas y asegurando que skipped y shuffle sean 0/1.\n")

cat_cols = ['platform', 'reason_start', 'reason_end']
for col in cat_cols:
    df[col] = df[col].astype('category')

df['shuffle_num'] = df['shuffle'].fillna(0).astype(int)
df['skipped_num'] = df['skipped'].fillna(0).astype(int)

print("Ejemplo de shuffle_num y skipped_num:")
display(df[['shuffle', 'shuffle_num', 'skipped', 'skipped_num']].head())


# 3.5 Crear columnas adicionales: hour y weekday
print("\n3.5 → Añadiendo nuevas columnas: 'hour' (hora del día) y 'weekday' (día de la semana 0–6).\n")
df['hour'] = df['ts'].dt.hour
df['weekday'] = df['ts'].dt.weekday

display(df[['ts', 'hour', 'weekday']].head())


## 4. Análisis exploratorio

## 4.1 Estadísticas globales

In [ ]:
print("\n=== 4. ANÁLISIS EXPLORATORIO ===")
print("=== 4.1 ESTADÍSTICAS GLOBALES ===")

# Total de registros
print("\n4.1.1 → Número total de reproducciones registradas en el dataset.\n"
      "Cada fila representa una reproducción de una canción.")
total_registros = len(df)
print("Total de registros:", total_registros)


# Estadísticas de minutes_played
print("\n4.1.2 → Estadísticas básicas de 'minutes_played'.\n"
      "Incluye mínimo, máximo, media, desviación estándar y percentiles.\n")
display(df['minutes_played'].describe())


# Tracks saltados vs completados
print("\n4.1.3 → ¿Cuántos tracks fueron saltados y cuántos no?\n"
      "skipped_num = 1 significa que la reproducción terminó en un salto.")
display(df['skipped_num'].value_counts())


# Sesiones con shuffle activado
print("\n4.1.4 → Número de reproducciones con shuffle activado o desactivado.\n"
      "shuffle_num = 1 indica que el modo aleatorio estaba activado.")
display(df['shuffle_num'].value_counts())


## 4.2 Distribución por artista / álbum / pista

In [ ]:
print("\n=== 4.2 DISTRIBUCIÓN POR ARTISTA / ÁLBUM / PISTA ===")

# 4.2.1 Artistas
print("\n4.2.1 → Estadísticas por ARTISTA.\n"
      "Para cada artista se muestra:\n"
      "- Número de reproducciones (count)\n"
      "- Minutos totales escuchados (sum)\n"
      "- Minutos promedio por reproducción (mean)\n")

artist_stats = df.groupby('artist_name').agg({
    'track_name': 'count',
    'minutes_played': ['sum', 'mean']
}).sort_values(('minutes_played', 'sum'), ascending=False)

print("Top 10 artistas con más tiempo escuchado:")
display(artist_stats.head(10))


# 4.2.2 Álbumes
print("\n4.2.2 → Álbumes más escuchados por tiempo total (en minutos).\n")

album_stats = df.groupby('album_name')['minutes_played'] \
                .sum().sort_values(ascending=False)

print("Top 10 álbumes más escuchados:")
display(album_stats.head(10))


# 4.2.3 Pistas
print("\n4.2.3 → Pistas más escuchadas por tiempo total.\n")

track_stats = df.groupby('track_name')['minutes_played'] \
                .sum().sort_values(ascending=False)

print("Top 10 canciones más escuchadas:")
display(track_stats.head(10))


## 4.3 Análisis temporal

In [ ]:
print("\n=== 4.3 ANÁLISIS TEMPORAL ===")

# 4.3.1 Escucha por hora del día
print("\n4.3.1 → Minutos escuchados según la HORA DEL DÍA (0–23).\n"
      "Esto permite ver en qué horas escuchas más música: mañana, tarde o noche.\n")

escucha_por_hora = df.groupby('hour')['minutes_played'].sum()
display(escucha_por_hora)


# 4.3.2 Escucha por día de la semana
print("\n4.3.2 → Minutos escuchados por DÍA DE LA SEMANA (0=Lunes, 6=Domingo).\n"
      "Sirve para ver si escuchas más entre semana o el fin de semana.\n")

escucha_por_dia = df.groupby('weekday')['minutes_played'].sum()
display(escucha_por_dia)


# 4.3.3 Comparación por plataforma
print("\n4.3.3 → Promedio de minutos escuchados según la PLATAFORMA.\n"
      "Ej.: Android, Web player, Windows…\n"
      "Ayuda a saber desde qué dispositivo escuchas sesiones más largas.\n")

comparacion_platform = df.groupby('platform')['minutes_played'].mean()
display(comparacion_platform)


# 4.3.4 Comparación según shuffle activado
print("\n4.3.4 → Tiempo medio escuchado según si SHUFFLE está activado.\n"
      "shuffle_num = 1 significa modo aleatorio activado.\n")

comparacion_shuffle = df.groupby('shuffle_num')['minutes_played'].mean()
display(comparacion_shuffle)


# 4.3.5 Relación entre shuffle y saltos
print("\n4.3.5 → Proporción de saltos (skipped_num) según uso de SHUFFLE.\n"
      "Sirve para ver si el modo aleatorio provoca más saltos.\n")

skip_por_shuffle = df.groupby('shuffle_num')['skipped_num'].mean()
display(skip_por_shuffle)


## 4.4 Comportamiento de saltos (skipped)

In [ ]:
print("\n=== 4.4 COMPORTAMIENTO DE SALTOS (SKIPPED) ===")

# 4.4.1 Porcentaje de tracks saltados
print("\n4.4.1 → Porcentaje total de canciones que han sido saltadas.\n"
      "Como 'skipped_num' es 0=no salto y 1=salto, la media indica el porcentaje real.")

porcentaje_skips = df['skipped_num'].mean() * 100
print(f"Porcentaje de saltos: {porcentaje_skips:.2f} %")


# 4.4.2 Saltos según reason_start
print("\n4.4.2 → Proporción de saltos agrupados por 'reason_start'.\n"
      "Muestra qué motivo de inicio está más asociado a que termines saltando la canción.")

skip_por_start = df.groupby('reason_start')['skipped_num'].mean().sort_values(ascending=False)
display(skip_por_start)


# 4.4.3 Saltos según reason_end
print("\n4.4.3 → Proporción de saltos agrupados por 'reason_end'.\n"
      "Indica qué motivo de finalización ocurre más cuando se produce un salto (por ejemplo: nextbtn).")

skip_por_end = df.groupby('reason_end')['skipped_num'].mean().sort_values(ascending=False)
display(skip_por_end)


# 4.4.4 Diferencias de minutos entre saltados vs no saltados
print("\n4.4.4 → Minutos medios escuchados según si se saltó o no la canción.\n"
      "Esto permite saber si saltas rápidamente las canciones o si las escuchas un tiempo antes de saltarlas.")

diferencia_tiempo = df.groupby('skipped_num')['minutes_played'].mean()
display(diferencia_tiempo)


## 5. Uso de NumPy para análisis auxiliar

In [ ]:
print("\n=== 5. USO DE NUMPY PARA ANÁLISIS AUXILIAR ===")

# 5.1 Percentiles
print("\n5.1 → Cálculo de percentiles de 'minutes_played'.\n"
      "Los percentiles permiten entender cómo se distribuye el tiempo escuchado.\n"
      "Ejemplos:\n"
      "- P25: 25% de las reproducciones duran menos que ese valor.\n"
      "- P50 (mediana): duración típica.\n"
      "- P95: solo el 5% de las reproducciones superan este valor.\n")

minutes = df['minutes_played'].to_numpy()
percentiles = np.percentile(minutes, [25, 50, 75, 95])

print("Percentiles 25, 50, 75, 95:")
print(percentiles)


# 5.2 Histograma
print("\n5.2 → Histograma de distribución del tiempo reproducido.\n"
      "El histograma divide los minutos en intervalos y cuenta cuántas canciones caen en cada rango.\n"
      "Sirve para identificar si la mayoría de reproducciones son cortas, largas o variadas.\n")

hist, bins = np.histogram(minutes, bins=20)

print("\nFrecuencias por intervalo:")
print(hist)

print("\nLímites de cada intervalo (bins):")
print(bins)


# 5.3 Correlación entre minutos escuchados y probabilidad de salto
print("\n5.3 → Correlación entre 'minutes_played' y 'skipped_num'.\n"
      "La correlación indica si existe relación entre el tiempo que escuchas una canción y la probabilidad de saltarla.\n"
      "Valores típicos:\n"
      "- Cerca de 0 → No hay relación.\n"
      "- Positivo → Cuanto más escuchas, más la saltas.\n"
      "- Negativo → Cuanto más dura la escucha, menos se salta.\n")

skipped_array = df['skipped_num'].to_numpy()
correlacion = np.corrcoef(minutes, skipped_array)

print("\nMatriz de correlación:")
print(correlacion)


## 6. Preguntas para los alumnos


1. ¿Cuál artista acumula más tiempo de escucha en total? ¿Y cuál pista?
    El artista con más tiempo total de escucha es The Beatles, con 20.169 minutos aproximadamente.
    La pista con más tiempo reproducido es “Ode To The Mets”, con unos 1123 minutos escuchados.

2. ¿En qué hora del día y en qué día de la semana se realiza la mayor parte de la escucha? ¿Hay diferencias entre plataformas o uso de shuffle?
    La hora del día donde más escuchas música es las 18:00, seguida muy de cerca por las 17:00 y 20:00.
    El día de la semana con mayor escucha es el viernes, seguido del martes.

    Respecto al uso:
        Cuando shuffle está apagado (0) las escuchas duran más (2.75 min de media).
        Con shuffle activado (1) las reproducciones tienden a ser más cortas (1.92 min).
        Esto sugiere que escuchas canciones más completas cuando eliges las canciones manualmente (sin shuffle).

3. ¿Cuál es el porcentaje de saltos (skipped == TRUE)? ¿Qué motivos de inicio (reason_start) o fin (reason_end) están más asociados a saltos?
    El porcentaje total de canciones saltadas es 5.23%.

    Los motivos con mayor relación con saltos son:
        Motivo de inicio con más saltos: popup
        (Cuando la reproducción empieza debido a un popup, casi siempre acaba en salto)
        Motivo de fin más asociado a saltos: appload
        (Normalmente canciones que terminan durante la carga de la aplicación)

4. ¿Existen pistas que suelen reproducirse más tiempo (high minutes_played) y rara vez se saltan? ¿Cuáles son y por qué crees que lo son?
    Según los tiempos de reproducción medios (y viendo que el porcentaje de saltos general es muy bajo), hay canciones como:
    “Ode To The Mets” y seguramente otras de The Beatles, ya que acumulan mucho tiempo total

    que muestran un patrón de:
        reproducirse durante mucho tiempo
        ser muy pocas veces saltadas
